In [14]:
import requests
import pandas as pd
from pprint import pprint

url = 'http://spreadtrends.com/api/nfl/games/2020?format=json'

game_data = requests.get(url).json()

# create dataframe of results using pandas json normalize
df = pd.json_normalize(game_data, record_path = ['results'])

# create dataframe of the columns we want to analyze

weather_df = df[['gameId', 'season', 'type','surface', 'temperature', 'conditions', 'windSpeed']]

# filter out all game types other than 'Regular'
weather_df = weather_df.loc[weather_df['type'] == 'Regular']



In [16]:
# Player stats for each game played

game_stat_url = 'http://spreadtrends.com/api/nfl/games/2020/1/stats?format=json'

stat_results = requests.get(game_stat_url).json()

# create dataframe of results
game_stat_df = pd.json_normalize(stat_results, record_path = ['results'])
game_stat_df.head()



# Get player stat data for each game
# QB, RB, WR, K

qb_df = game_stat_df.loc[game_stat_df['position'] == 'QB']
qb_df.head()

rb_df = game_stat_df.loc[game_stat_df['position'] == 'RB']
rb_df.head()

wr_df = game_stat_df.loc[game_stat_df['position'] == 'WR']

k_df = game_stat_df.loc[game_stat_df['position'] == 'K']

# Merge game stat data with player stat data

print(qb_df.columns)

Index(['gameId', 'season', 'week', 'lastName', 'firstName', 'position',
       'uniformNumber', 'opponent', 'team', 'att', 'comp', 'passYds',
       'passTds', 'ints', 'sacks', 'sackYds', 'rush', 'rushYds', 'rushTds',
       'rec', 'recYds', 'recTds'],
      dtype='object')


In [4]:
# QB Rating cacluation

A = ((qb_df['comp'] / qb_df['att'])-.3)*5
B = ((qb_df['passYds'] / qb_df['att'])-3)*.25
C = ((qb_df['passTds'] / qb_df['att']))*20
D = (2.375 - ((qb_df['ints'] / qb_df['att'])*25))

qb_rating = ((A + B + C + D)/6)*100

In [5]:
qb_df['QB Rating'] = qb_rating

<ipython-input-5-289a2959c638>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qb_df['QB Rating'] = qb_rating


In [6]:
qb_df.head()

,gameId,season,week,lastName,firstName,position,uniformNumber,opponent,team,att,...,ints,sacks,sackYds,rush,rushYds,rushTds,rec,recYds,recTds,QB Rating
1,4936,2020,1,Garoppolo,J,QB,10,Cardinals,49ers,33,...,0,3,16,1,9,0,0,0,0,102.967172
10,4932,2020,1,Trubisky,Mitchell,QB,10,Lions,Bears,36,...,0,1,28,3,26,0,0,0,0,104.166667
22,4935,2020,1,Burrow,Joe,QB,9,Chargers,Bengals,36,...,1,3,20,8,46,1,0,0,0,66.087963
32,4926,2020,1,Allen,Josh,QB,17,Jets,Bills,46,...,0,3,6,14,57,1,0,0,0,104.619565
42,4940,2020,1,Lock,Drew,QB,3,Titans,Broncos,33,...,0,0,0,3,5,0,0,0,0,95.012626


In [13]:
# calculate yards per rush attempt

yards_per_rush = rb_df['rushYds'] / rb_df['rush']

rb_df['yards per rush'] = yards_per_rush

rb_df.head()

<ipython-input-13-90a6905f4e09>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rb_df['yards per rush'] = yards_per_rush


,gameId,season,week,lastName,firstName,position,uniformNumber,opponent,team,att,...,ints,sacks,sackYds,rush,rushYds,rushTds,rec,recYds,recTds,yards per rush
2,4936,2020,1,Mostert,R,RB,43,Cardinals,49ers,0,...,0,0,0,15,56,0,4,95,1,3.733333
3,4936,2020,1,McKinnon,J,RB,31,Cardinals,49ers,0,...,0,0,0,3,24,0,3,20,1,8.000000
4,4936,2020,1,Coleman,T,RB,26,Cardinals,49ers,0,...,0,0,0,4,18,0,1,6,0,4.500000
11,4932,2020,1,Montgomery,David,RB,32,Lions,Bears,0,...,0,0,0,13,64,0,1,10,0,4.923077
12,4932,2020,1,Cohen,Tarik,RB,29,Lions,Bears,0,...,0,0,0,7,41,0,2,6,0,5.857143


In [11]:
#merge QB stats with weather data

qb_merge = qb_df.merge(weather_df, how='left', on='gameId')

qb_merge.head()

,gameId,season_x,week,lastName,firstName,position,uniformNumber,opponent,team,att,...,rec,recYds,recTds,QB Rating,season_y,type,surface,temperature,conditions,windSpeed
0,4936,2020,1,Garoppolo,J,QB,10,Cardinals,49ers,33,...,0,0,0,102.967172,2020,Regular,Grass,66.0,haze,6.00
1,4932,2020,1,Trubisky,Mitchell,QB,10,Lions,Bears,36,...,0,0,0,104.166667,2020,Regular,Turf-Dome,71.0,partly cloudy,14.00
2,4935,2020,1,Burrow,Joe,QB,9,Chargers,Bengals,36,...,0,0,0,66.087963,2020,Regular,Turf,83.0,partly cloudy,6.00
3,4926,2020,1,Allen,Josh,QB,17,Jets,Bills,46,...,0,0,0,104.619565,2020,Regular,Grass,67.0,cloudy,15.00
4,4940,2020,1,Lock,Drew,QB,3,Titans,Broncos,33,...,0,0,0,95.012626,2020,Regular,Grass,74.0,clear sky,1.01


In [30]:
# loop through all weeks in the season and create data frame with stats

qb_stats_data_df = pd.DataFrame([])

for x in range(1,19):
    stats_url = f'http://spreadtrends.com/api/nfl/games/2020/{x}/stats?format=json'
    stats_results = requests.get(stats_url).json()
    temp_df = pd.json_normalize(stats_results, record_path = ['results'])
    qb_stats_data_df = stats_data_df.append(temp_df)
    
#create a csv file
qb_stats_data_df.to_csv('game_stats.csv')


In [31]:
qb_stats_data_df

,gameId,season,week,lastName,firstName,position,uniformNumber,opponent,team,att,...,passTds,ints,sacks,sackYds,rush,rushYds,rushTds,rec,recYds,recTds
0,4936,2020,1,Juszczyk,K,FB,40,Cardinals,49ers,0,...,0,0,0,0,1,7,0,1,41,0
1,4936,2020,1,Garoppolo,J,QB,10,Cardinals,49ers,33,...,2,0,3,16,1,9,0,0,0,0
2,4936,2020,1,Mostert,R,RB,43,Cardinals,49ers,0,...,0,0,0,0,15,56,0,4,95,1
3,4936,2020,1,McKinnon,J,RB,31,Cardinals,49ers,0,...,0,0,0,0,3,24,0,3,20,1
4,4936,2020,1,Coleman,T,RB,26,Cardinals,49ers,0,...,0,0,0,0,4,18,0,1,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,5470,2020,18,Sims,Cam,WR,89,Buccaneers,Washington,0,...,0,0,0,0,0,0,0,7,104,0
116,5470,2020,18,McLaurin,Terry,WR,17,Buccaneers,Washington,0,...,0,0,0,0,0,0,0,6,75,0
117,5470,2020,18,Sims Jr.,Steven,WR,15,Buccaneers,Washington,0,...,0,0,0,0,1,4,0,3,33,1
118,5470,2020,18,McKissic,J.D.,WR,30,Buccaneers,Washington,0,...,0,0,0,0,2,5,1,2,8,0


In [32]:
combined_qb = qb_stats_data_df.merge(weather_df, how='left', on='gameId')

combined_qb

,gameId,season_x,week,lastName,firstName,position,uniformNumber,opponent,team,att,...,rushTds,rec,recYds,recTds,season_y,type,surface,temperature,conditions,windSpeed
0,4936,2020,1,Juszczyk,K,FB,40,Cardinals,49ers,0,...,0,1,41,0,2020.0,Regular,Grass,66.0,haze,6.0
1,4936,2020,1,Garoppolo,J,QB,10,Cardinals,49ers,33,...,0,0,0,0,2020.0,Regular,Grass,66.0,haze,6.0
2,4936,2020,1,Mostert,R,RB,43,Cardinals,49ers,0,...,0,4,95,1,2020.0,Regular,Grass,66.0,haze,6.0
3,4936,2020,1,McKinnon,J,RB,31,Cardinals,49ers,0,...,0,3,20,1,2020.0,Regular,Grass,66.0,haze,6.0
4,4936,2020,1,Coleman,T,RB,26,Cardinals,49ers,0,...,0,1,6,0,2020.0,Regular,Grass,66.0,haze,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5731,5470,2020,18,Sims,Cam,WR,89,Buccaneers,Washington,0,...,0,7,104,0,NaN,NaN,NaN,NaN,NaN,NaN
5732,5470,2020,18,McLaurin,Terry,WR,17,Buccaneers,Washington,0,...,0,6,75,0,NaN,NaN,NaN,NaN,NaN,NaN
5733,5470,2020,18,Sims Jr.,Steven,WR,15,Buccaneers,Washington,0,...,0,3,33,1,NaN,NaN,NaN,NaN,NaN,NaN
5734,5470,2020,18,McKissic,J.D.,WR,30,Buccaneers,Washington,0,...,1,2,8,0,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
combined_qb.to_csv('QBstats_weatherData.csv')